In [1]:
import math
import os
import shutil
import time
from pathlib import Path

import matplotlib.pyplot as plt
import streamlit as st
import numpy as np
import numba
import pyvista as pv

2022-10-12 12:29:38.976 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [3]:
from LightTransportSimulator.light_transport.src.scene import Scene, Light
from LightTransportSimulator.light_transport.src.primitives import Triangle, PreComputedTriangle
from LightTransportSimulator.light_transport.src.cornell_box import get_cornell_box, get_floor
from LightTransportSimulator.light_transport.src.colors import *
from LightTransportSimulator.light_transport.src.render_old import render_scene

In [4]:
#load object
mesh = pv.Cone(radius=2, center=[0,0,0], height=5)
tri = mesh.triangulate()
points = np.ascontiguousarray(tri.points)
faces = tri.faces.reshape((-1,4))[:, 1:4]
vertices = np.ascontiguousarray(points[faces], dtype=np.float64)
len(vertices)

10

In [5]:
#set scene depths

xmax, ymax, zmax = points.max(axis=0)
xmin, ymin, zmin = points.min(axis=0)

padding = 5
depth = abs(max(xmax, ymax, zmax)) + padding
x_depth = abs(xmax) + 10
y_depth = abs(ymax) + 1
z_depth = abs(zmax) + 10
xmax, ymax, zmax, depth

(2.5, 2.0, 1.7320508, 7.5)

In [6]:
# empty object list
objects = numba.typed.List()

In [7]:
start_id = len(objects)
surface_color = WHITE_2
left_wall_color = GREEN
right_wall_color = RED
surface_mat = Material(color=surface_color, shininess=30, reflection=0.1, ior=1.5210, transmission=1) # calcium sulphate
left_wall_mat = Material(color=left_wall_color, shininess=30, reflection=0.1, ior=1.5210, transmission=1)
right_wall_mat = Material(color=right_wall_color, shininess=30, reflection=0.1, ior=1.5210, transmission=1)

objects = get_cornell_box(depth, surface_mat, left_wall_mat, right_wall_mat, start_id)

In [8]:
id = len(objects)+1
for v in vertices:
    a,b,c = np.append(v[0], 1),np.append(v[1], 1),np.append(v[2], 1)
    id+=1
    triangle = PreComputedTriangle(id=id,
                                   vertex_1=np.ascontiguousarray(a, dtype=np.float64),
                                   vertex_2=np.ascontiguousarray(b, dtype=np.float64),
                                   vertex_3=np.ascontiguousarray(c, dtype=np.float64),
                                   material=GLASS_MAT)
    objects.append(triangle)

objects

ListType[instance.jitclass.PreComputedTriangle#2adbcada410<type:int64,id:int64,vertex_1:array(float64, 1d, C),vertex_2:array(float64, 1d, C),vertex_3:array(float64, 1d, C),centroid:array(float64, 1d, C),material:instance.jitclass.Material#2adbcb1a3b0<color:instance.jitclass.Color#2adbcb3aa70<ambient:array(float64, 1d, A),diffuse:array(float64, 1d, A),specular:array(float64, 1d, A)>,shininess:float64,reflection:float64,ior:float64,is_diffuse:bool,is_mirror:bool,emission:float64,transmission:float64>,is_light:bool,edge_1:array(float64, 1d, C),edge_2:array(float64, 1d, C),normal:array(float64, 1d, C),num:float64,transformation:array(float64, 2d, A)>]([<numba.experimental.jitclass.boxing.PreComputedTriangle object at 0x000002ADC0E9B940>, <numba.experimental.jitclass.boxing.PreComputedTriangle object at 0x000002ADC0E9BE50>, <numba.experimental.jitclass.boxing.PreComputedTriangle object at 0x000002ADC0E9B940>, <numba.experimental.jitclass.boxing.PreComputedTriangle object at 0x000002ADC0E9BE

In [9]:
# global light source

source_mat = Material(color=WHITE, shininess=1, reflection=0.9, ior=1.5)

l_x = 1
l_y = depth+0.1
l_z = 1

id = len(objects)+1

light_1 = PreComputedTriangle(id=id,
                              vertex_1=np.array([-l_x, l_y, -l_z, 1], dtype=np.float64),
                              vertex_2=np.array([l_x, l_y, l_z, 1], dtype=np.float64),
                              vertex_3=np.array([-l_x, l_y, l_z, 1], dtype=np.float64),
                              material=source_mat,
                              is_light=True)
id += 1

light_2 = PreComputedTriangle(id=id,
                              vertex_1=np.array([l_x, l_y, l_z, 1], dtype=np.float64),
                              vertex_2=np.array([-l_x, l_y, -l_z, 1], dtype=np.float64),
                              vertex_3=np.array([l_x, l_y, -l_z, 1], dtype=np.float64),
                              material=source_mat,
                              is_light=True)


light_samples = 10
a = np.random.uniform(0,1,light_samples)
b = np.random.uniform(1,0,light_samples)

light_sources = numba.typed.List()

for x in range(light_samples):
    tp1 = light_1.vertex_1 * (1-math.sqrt(a[x])) + light_1.vertex_2 * (math.sqrt(a[x])*(1-b[x])) + light_1.vertex_3 * (b[x]*math.sqrt(a[x]))
    l1 = Light(source=tp1, material=source_mat)
    light_sources.append(l1)
    tp2 = light_2.vertex_1 * (1-math.sqrt(a[x])) + light_2.vertex_2 * (math.sqrt(a[x])*(1-b[x])) + light_2.vertex_3 * (b[x]*math.sqrt(a[x]))
    l2 = Light(source=tp1, material=source_mat)
    light_sources.append(l2)
    print(tp1, tp2)

light_sources

[-0.86172049  7.6        -0.53692063  1.        ] [0.86172049 7.6        0.53692063 1.        ]
[0.4715552  7.6        0.48375303 1.        ] [-0.4715552   7.6        -0.48375303  1.        ]
[-0.20460959  7.6         0.05560446  1.        ] [ 0.20460959  7.6        -0.05560446  1.        ]
[-0.0903753   7.6         0.19074159  1.        ] [ 0.0903753   7.6        -0.19074159  1.        ]
[0.27923001 7.6        0.29533602 1.        ] [-0.27923001  7.6        -0.29533602  1.        ]
[-0.61018251  7.6        -0.24357509  1.        ] [0.61018251 7.6        0.24357509 1.        ]
[-0.1325838   7.6         0.62337036  1.        ] [ 0.1325838   7.6        -0.62337036  1.        ]
[-0.10659974  7.6         0.71557085  1.        ] [ 0.10659974  7.6        -0.71557085  1.        ]
[-0.66615974  7.6         0.51304701  1.        ] [ 0.66615974  7.6        -0.51304701  1.        ]
[-0.66649625  7.6         0.37253113  1.        ] [ 0.66649625  7.6        -0.37253113  1.        ]


ListType[instance.jitclass.Light#2adbcb18b50<source:array(float64, 1d, A),material:instance.jitclass.Material#2adbcb1a3b0<color:instance.jitclass.Color#2adbcb3aa70<ambient:array(float64, 1d, A),diffuse:array(float64, 1d, A),specular:array(float64, 1d, A)>,shininess:float64,reflection:float64,ior:float64,is_diffuse:bool,is_mirror:bool,emission:float64,transmission:float64>>]([<numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEB400>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEBB50>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEB400>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEBB50>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEB400>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEBB50>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEB400>, <numba.experimental.jitclass.boxing.Light object at 0x000002ADC0FEBB50>, <numba.experimental.jitclass.boxing.Li

In [10]:
objects.append(light_1)
objects.append(light_2)

In [11]:
# setup scene

#camera
camera = np.array([0, 0, depth+0.5, 1], dtype=np.float64)

#screen
width=400
height=400
max_depth=3

#scene
scene = Scene(camera=camera, lights=light_sources, width=width, height=height, max_depth=max_depth, depth=depth)

In [12]:
from LightTransportSimulator.light_transport.src.primitives import AABB
# min_point=np.array([scene.left, scene.bottom, -scene.depth], dtype=np.float64)
# max_point=np.array([scene.right, scene.top, scene.depth], dtype=np.float64)
min_point=np.array([xmin, ymin, zmin], dtype=np.float64)
max_point=np.array([xmax, ymax, zmax], dtype=np.float64)
box = AABB(min_point=min_point, max_point=max_point)
box

In [13]:
from LightTransportSimulator.light_transport.src.bvh import BVH

bvh = BVH()
bvh.insert(objects, box)

In [ ]:
start = time.time()
image = render_scene(scene, bvh.top)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

1
101
30135151
102
2


352
30252

103
3533

53
3034

35410454

304
5

35530510555



6
7
56356
106
306

8
57
307357910758




59308
10
108358


11
309
12
35960109310



13
6114110
360

311

3611531262
111



6311216313362




17
64
113314

18363

65
315
1146619364316




3173652067115




1166831821366




22
36711769319
23



3207011836824




2571
321369119


72

1203222673370




37174
12127323
75



3722876122324




7712332529373




7837412432630




3271253753179




80328
12637632
329



8137733330127




1283318237834




12983332379
35



333
130380
3684

334

131381

335
37
85
1323363838286




13387337383



39
38488338
40134



339
8913538541340




3863414290136




9143342387



3889234344



9338945344



3453904694



39134695


47
3929634748



97393

49348

3945098


349
39599350


100396

397
398
399
400
137
251
151
138
252
139
253
152
140
254
141
255
142
153
256
143
257
144
258
145
259
154
146
260
147
261
148
262
263
149
264
265
266
267
268
269
270
271
272
273
274


In [ ]:
plt.imshow(image)